In [11]:
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [ ]:
from cmdstanpy import CmdStanModel

In [12]:
import pandas as pd
import numpy as np
import seaborn as sns
from datetime import datetime
import os

In [13]:
os.chdir('/Users/sahoko/GitHub/WFP-GP-temporary/')
country_list = os.listdir('Data/Foini2023/output_data')
country_list.remove('.DS_Store')
country_list

['Syria', 'Burkina-Faso', 'Yemen', 'Mali', 'Nigeria', 'Niger', 'Cameroon']

In [14]:
# Choose country from country_list ['Syria', 'Burkina-Faso', 'Yemen', 'Mali', 'Nigeria', 'Niger', 'Cameroon']
country = 'Burkina-Faso'

In [15]:
df_info_ctry = pd.read_csv('Data/Foini2023/Missing-data-info/country-info.csv', index_col=0)

In [16]:
file = 'Data/Foini2023/output_data/'+ country +'/' + country +'.csv'
df = pd.read_csv(file, header=[0,1])
st = df.index[df['AdminStrata']['Indicator']== df_info_ctry.loc[country]['startday']][0]
en = df.index[df['AdminStrata']['Indicator']== df_info_ctry.loc[country]['lastday']][0]
df = df.iloc[st:en]

In [17]:
n_row = len(df)
region_list = df.columns.get_level_values(0)[1:].to_list()
column_list= df.columns.get_level_values(1)[1:].to_list()
region_list = (list(dict.fromkeys(region_list)))
column_list = (list(dict.fromkeys(column_list)))
df_long = pd.DataFrame(columns=column_list)
for region in region_list:
    df_long = pd.concat([df_long, df[region]], axis=0)
df_long['region'] = list(np.repeat(region_list,len(df)))
df_long['Datetime'] = list(pd.to_datetime(df['AdminStrata']['Indicator']))*len(region_list)
df_long

,1 Month Anomaly Rainfalls (%),3 Months Anomaly Rainfalls (%),Area,Code,FCG,Fatalities,Lat,Lon,NDVI,NDVI Anomaly,Population,Price cereals and tubers,Rainfalls (mm),Ramadan,Waterways,rCSI,region,Datetime
615,NaN,NaN,2.866733,0,30.921700,4.0,12.535927,-3.488905,NaN,NaN,1816310,NaN,NaN,0.0,9.290487,19.499500,Boucle-Du-Mouhoun,2019-09-08
616,NaN,NaN,2.866733,0,31.103930,4.0,12.535927,-3.488905,NaN,NaN,1816310,NaN,NaN,0.0,9.290487,19.567380,Boucle-Du-Mouhoun,2019-09-09
617,112.112436,107.023646,2.866733,0,32.030300,4.0,12.535927,-3.488905,0.598706,97.420785,1816310,NaN,69.165,0.0,9.290487,20.400680,Boucle-Du-Mouhoun,2019-09-10
618,NaN,NaN,2.866733,0,32.167300,4.0,12.535927,-3.488905,NaN,NaN,1816310,NaN,NaN,0.0,9.290487,20.405580,Boucle-Du-Mouhoun,2019-09-11
619,NaN,NaN,2.866733,0,31.770050,4.0,12.535927,-3.488905,NaN,NaN,1816310,NaN,NaN,0.0,9.290487,19.238530,Boucle-Du-Mouhoun,2019-09-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1564,NaN,NaN,1.363722,12,45.410901,3.0,10.475215,-3.237191,NaN,NaN,796535,NaN,NaN,14.0,6.444829,14.366029,Sud-Ouest,2022-04-14
1565,NaN,NaN,1.363722,12,44.728667,3.0,10.475215,-3.237191,NaN,NaN,796535,NaN,NaN,15.0,6.444829,12.660446,Sud-Ouest,2022-04-15
1566,NaN,NaN,1.363722,12,42.369553,NaN,10.475215,-3.237191,NaN,NaN,796535,NaN,NaN,16.0,6.444829,12.285583,Sud-Ouest,2022-04-16
1567,NaN,NaN,1.363722,12,42.755724,NaN,10.475215,-3.237191,NaN,NaN,796535,NaN,NaN,17.0,6.444829,12.757992,Sud-Ouest,2022-04-17


In [ ]:
# making Y to long format, then transform y 
# change longitude latitude -> easting northing 
# change timestamp -> {1,2,...}  

